In [1]:
from idCrawler import getUserProfile
from pymongo import MongoClient

In [40]:
def patientsReader_bipolar(file_name,keywords=["borderline","bpd"]):
    f = open(file_name, 'r')
    patient_ids = {}
    valid_conditions = ['o','p','c']
    for line in f:
        try:
            segments = line.strip().split("\t")
            condition = segments[0].lower()
            description = segments[1].lower()
            screen_name = segments[2]
            user_id = segments[3]
            
        except:
            print(line)
            return patient_ids
        if condition in valid_conditions:
            if any(keyword in description for keyword in keywords):
                patient_ids[user_id] = 'c'
            else:
                patient_ids[user_id] = 'o'
       
        
    return patients_ids

In [59]:
def patientsReader_BPD(file_name, keywords = ["bipolar"]):
    f = open(file_name, 'r')
    patient_ids = {}
    for line in f:
        try:
            segments = line.strip().split("\t")
            condition = segments[0]
            description = segments[1].lower()
            user_id = segments[2]
            
        except:
            print(line)
            return patient_ids
        if condition == 'o' or condition == 'O' or condition == 'p':
            if any(keyword in description for keyword in keywords):
                patient_ids[user_id] = 'c'
            else:
                patient_ids[user_id] = 'o'
            
        
    return patient_ids
def writeUsers(tweets, collectionName):
    collection = MongoClient("localhost", 27017)["patients"][collectionName]
    collection.insert(tweets)
    print("{} tweets inserted".format(len(tweets)))

In [39]:
file_location = "tagged_bpd_descriptions.tsv"
BPD_ids = patientsReader_BPD(file_location)
comorbidities_BPD = []
for BPD_id, condition in BPD_ids.items():
    if condition == 'c':
        comorbidities_BPD.append(BPD_id)
print(len(comorbidities_BPD))

78


In [43]:
file_location = "bipolar_profile.tsv"
bipolar_ids = patientsReader_bipolar(file_location)
comorbidities_bipolar = []
for bipolar_id, condition in bipolar_ids.items():
    if condition == 'c':
        comorbidities_bipolar.append(bipolar_id)
print(len(comorbidities_bipolar))

	bipolar disorder sufferer and all round bad person, take no heed in what I tweet!	manc_swag	3097925127

76


{'1040789676': 'o',
 '1045167511': 'o',
 '106416595': 'c',
 '1073707561': 'o',
 '1074974424': 'o',
 '107823855': 'o',
 '1080026598': 'c',
 '108637252': 'o',
 '1087659283': 'o',
 '1096422758': 'o',
 '1104536850': 'o',
 '1106347554': 'o',
 '1107458226': 'o',
 '110833250': 'o',
 '1112651587': 'o',
 '1115036520': 'o',
 '1118469732': 'o',
 '1147083032': 'o',
 '1157997235': 'o',
 '116439735': 'o',
 '116822107': 'o',
 '1171197608': 'o',
 '1171747466': 'o',
 '1181680603': 'o',
 '1198315274': 'o',
 '1202544439': 'o',
 '1222900213': 'o',
 '1223074080': 'o',
 '1226438646': 'o',
 '123687707': 'o',
 '1240215072': 'c',
 '1247935578': 'c',
 '1250889727': 'o',
 '1257706351': 'o',
 '1270867711': 'o',
 '1287172507': 'o',
 '13009262': 'o',
 '130692007': 'o',
 '1316415847': 'o',
 '1331857429': 'c',
 '1335363289': 'o',
 '1341705313': 'o',
 '1342000794': 'o',
 '134246394': 'o',
 '1344581366': 'o',
 '1344779467': 'o',
 '1352182662': 'o',
 '1352465874': 'o',
 '1355518574': 'c',
 '1357902660': 'o',
 '135964180

In [54]:
BPD_clean = {}
comorbidity_tweets = {}
lost_ids = set()

BPD_cursor = MongoClient("localhost", 27017)['idea']["BPD_581_emotion"].find()
for tweet in BPD_cursor:
    tweet_id = tweet["id"]
    user_id = str(tweet["user"]["id"])
    if user_id in BPD_ids:
        condition = BPD_ids[user_id]
        if condition == 'c':
            comorbidity_tweets[tweet_id] = tweet
        else:
            BPD_clean[tweet_id] = tweet
    else:
        lost_ids.add(user_id)
        


    
print(len(comorbidity_tweets))
print(len(BPD_clean))

58439
303818


In [55]:
bipolar_clean = {}
#comorbidity_tweets = {}
lost_ids = set()

bipolar_cursor = MongoClient("localhost", 27017)['idea']["bipolar_emotion"].find()
for tweet in bipolar_cursor:
    tweet_id = tweet["id"]
    user_id = str(tweet["user"]["id"])
    if user_id in bipolar_ids:
        condition = bipolar_ids[user_id]
        if condition == 'c':
            comorbidity_tweets[tweet_id] = tweet
        else:
            bipolar_clean[tweet_id] = tweet
    else:
        lost_ids.add(user_id)
        


    
print(len(comorbidity_tweets))
print(len(bipolar_clean))

80866
279129


In [64]:
writeUsers(list(comorbidity_tweets.values()),"bb_mix")

80866 tweets inserted


In [56]:
overlap_n = 0
for user_id in bipolar_ids:
    if any(user_id == x for x in BPD_ids):
        overlap_n += 1
print(overlap_n)

61


In [53]:
len(lost_ids)

0

In [17]:
len(BPD_patietn_ids)

496

In [ ]:
keywords = ["bipolar"]
for profile in BPD_profiles:
    description = BPD_profiles["description"].lower()
    if keywords in description:
        BPD_profiles["condition"] = ["BPD","bipolar"]
    else:
        BPD_profiles["condition"] = ["BPD"]

In [ ]:
bipolar_patietn_ids = set()
bipolar_cursor = MongoClient("localhost", 27017)['idea']["bipolar_emotion"].find()
for tweet in bipolar_cursor:
    user_id = tweet["user"]["id"]
    bipolar_patietn_ids.add(user_id)
bipolar_profiles = []
for user_id in bipolar_patietn_ids:
    bipolar_profiles += getUserProfile(user_id=user_id)

    

In [ ]:
keywords = ["borderline","bpd"]
for profile in bipolar_profiles:
    description =bipolarD_profiles["description"].lower()
    if keywords in description:
        bipolar_profiles["condition"] = ["BPD","bipolar"]
    else:
        bipolar_profiles["condition"] = ["BPD"]